In [ ]:
import qnexus as qnx
import numpy as np

qnx.login()

In [38]:
import qnexus as qnx
import uuid

unique_suffix = uuid.uuid1()

project = qnx.projects.get_or_create("Helios-Samples")
qnx.context.set_active_project(project)

In [102]:
import networkx as nx
from collections import Counter
from guppylang import guppy
from guppylang.defs import GuppyFunctionDefinition
from guppylang.std.builtins import array, comptime, result
from guppylang.std.quantum import rx, rz, cx, h, measure_array, qubit
from guppylang.emulator import EmulatorResult
from guppylang.std.angles import angle, pi
import numpy as np
import os

In [105]:
# Define helper functions from FC-Experiments.ipynb
from collections import defaultdict, Counter
import numpy as np

def cost_maxcut(bitstring, weights):
    """
    Computes the cost of a given bitstring solution for the Max-Cut problem.
    
    Parameters:
    bitstring (str): A binary string representing a partition of the graph nodes.
    weights (dict): A dictionary where keys are edge tuples (i, j) and values are edge weights.
    
    Returns:
    float: The computed cost of the Max-Cut solution.
    """
    cost = 0
    for i, j in weights.keys():
        if bitstring[i] + bitstring[j] in ["10", "01"]:
            cost += weights[i, j]
    return cost

def objective_MaxCut(samples_dict, G, optimal):
    """
    Evaluates the performance of a quantum algorithm for the Max-Cut problem.
    
    Parameters:
    samples_dict (dict): A dictionary where keys are bitstrings, values are counts.
    G (networkx.Graph): The input weighted graph.
    optimal (str): The optimal bitstring solution.
    
    Returns:
    dict: Contains results, max_cut, approximation ratio r, and probability.
    """
    weights = {(i, j): (G[i][j]["weight"] if len(G[i][j]) != 0 else 1) for i, j in G.edges}
    max_cost = cost_maxcut(optimal, weights)
    
    results = []
    probability = 0
    
    for bitstring, counts in samples_dict.items():
        cost = cost_maxcut(bitstring, weights)
        r = cost / max_cost
        results.append([cost, r, counts])
        
        if abs(cost - max_cost) < 1e-6:
            probability += counts
        
        if cost > max_cost:
            print(f"Found better solution than optimal: {cost - max_cost}")
    
    results = np.array(results)
    shots = np.sum(results[:, 2])
    rT = np.sum(results[:, 0] * results[:, 2]) / (shots * max_cost)
    probability /= shots
    
    return {
        "results": np.array(results),
        "G": G,
        "weights": weights,
        "max_cut": max_cost,
        "r": rT,
        "probability": probability
    }

def random_samples(num_samples, n_qubits):
    """
    Generates random bitstring samples for a given number of qubits.

    Parameters:
    num_samples (int): The number of random bitstrings to generate.
    n_qubits (int): The number of qubits (length of each bitstring).

    Returns:
    dict: A dictionary where keys are randomly generated bitstrings 
          and values are their occurrence counts.
    """
    
    random_samples = defaultdict(int)  # Dictionary to store bitstrings and their counts

    # Generate random bitstrings and count their occurrences
    for _ in range(num_samples):
        bitstring = "".join(str(i) for i in np.random.choice([0, 1], n_qubits))  # Generate a random bitstring
        random_samples[bitstring] += 1  # Increment count for the generated bitstring

    return random_samples  # Return the dictionary of samples


In [29]:
def get_counts(shots: EmulatorResult) -> Counter[str]:
    """Counter treating all results from a shot as entries in a single bitstring"""
    counter_list = []
    for shot in shots:
        for e in shot:
            bitstring = "".join(str(k) for k in e[1])
            counter_list.append(bitstring)

    return dict(Counter(counter_list))

In [30]:
def qaoa(graph: nx.Graph, betas:list, gammas:list) -> GuppyFunctionDefinition:
    edges = list(graph.edges)
    num_layers = len(gammas)
    n_qb = graph.number_of_nodes()
    max_weight = max([graph[i][j]["weight"] for i, j in graph.edges])
    weights = [graph[i][j]["weight"]/max_weight for i, j in graph.edges]
    pi_num = float(np.pi)
    @guppy
    def main() -> None:
        qs = array(qubit() for _ in range(comptime(n_qb)))
        for i in range(len(qs)):
            h(qs[i])
        for layer_i in range(comptime(num_layers)):
            kk = 0
            for i, j in comptime(edges):
                cx(qs[i], qs[j])
                rz(qs[j], angle(2.0 * comptime(gammas)[layer_i] * comptime(weights)[kk] / comptime(pi_num)))
                cx(qs[i], qs[j])
                kk += 1
            for i in range(len(qs)):
                rx(qs[i], angle(- 2.0 * comptime(betas)[layer_i] / comptime(pi_num)))
        result("c", measure_array(qs))

    return main

In [165]:
res_i = np.load("/Users/alejomonbar/Documents/GitHub/LR-QAOA-QPU-Benchmarking/Data/H2-1/50_FC.npy", allow_pickle=True).item()
print(res_i["Deltas"][0])
{p:res_i["postprocessing"][res_i["Deltas"][0]][p][0]["r"] for p in res_i["ps"]}

0.2


{3: np.float64(0.846844859813084), 4: np.float64(0.8489869158878508)}

In [172]:
problems.keys()

dict_keys([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 100, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])

In [173]:
# Initialize results dictionary
results = {}
backend_name = "quantinuum_helios_1e"  # Backend name for saving results

# Problem parameters
nq = 98
problems = np.load("./Data/WMC_FC.npy", allow_pickle=True).item()
graph = problems[nq]["G"]
optimal_sol = problems[nq]["sol"]

# QAOA parameters
ps = [3]  # List of QAOA depths to test
shots = 50

# Store metadata in results
deltas = [0.2]
results["Deltas"] = deltas  # Store as list of tuples
results["G"] = graph
results["sections"] = 1
results["shots"] = shots
results["ps"] = ps
results["optimal"] = optimal_sol
results["total_qubits_used"] = list(range(nq))

# Test local emulator for small problems
programs = []
dict_results = []
for delta in deltas:
    for p in ps:
        gammas = [k * delta / p for k in range(1, p + 1)]
        betas = [(p - k + 1) * delta / p for k in range(1, p + 1)]
        graph_prog = qaoa(graph, betas, gammas)
        graph_prog_compiled = graph_prog.compile()
        programs.append(graph_prog_compiled)

        if nq <= 20:
            res = graph_prog.emulator(n_qubits=nq).with_shots(shots).run()
            samples = get_counts(res)
            dict_results.append(samples)
            



In [ ]:
nexus_programs = [qnx.hugr.upload(
    graph_prog_compiled, 
    name=f"qaoa-fc-{nq}q-{unique_suffix}"
) for graph_prog_compiled in programs]

prediction = qnx.hugr.cost(
    programs=nexus_programs, 
    n_shots=len(nexus_programs) * [shots]  # More shots for better statistics
)

In [176]:
config = qnx.models.HeliosConfig(
    system_name="Helios-1E",
    max_cost=np.ceil(prediction),
    emulator_config=qnx.models.HeliosEmulatorConfig(
        n_qubits=nq, 
        simulator=qnx.models.MatrixProductStateSimulator(chi=16)
    )
)

result_ref = qnx.start_execute_job(
    programs=nexus_programs,
    n_shots= len(ps) * [shots],  # Match the cost prediction
    backend_config=config,
    name=f"QAOA-FC-{nq}q-Helios-Emulator-{unique_suffix}",
)

In [177]:
def get_dict_results(job_result):
    # Process the results - extract bitstrings from QsysShot objects
    outcomes = []
    for shot_result in job_result:
        # Quantinuum returns measurements in index order: shot_result[0] is q0, etc.
        # Construct bitstring where bitstring[i] represents node i
        bitstring = shot_result.to_register_bits()["c"]
        outcomes.append(bitstring)
    samples_dict = dict(Counter(outcomes))
    return samples_dict

qnx.jobs.wait_for(result_ref)
job_results = [qnx.jobs.results(result_ref)[i].download_result() for i in range(len(nexus_programs))]

dict_results = [get_dict_results(job_result) for job_result in job_results]

Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.
Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.
Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.
Unknown OpType in BackendInfo: `%`, will omit from BackendInfo. Consider updating your pytket version.


In [178]:
results["samples"] = {
    delta: {
        p: {k: v for k, v in dict_results[i + nd * len(ps)].items()}
        for i, p in enumerate(results["ps"])
    }
    for nd, delta in enumerate(results["Deltas"])
    }
extra = ""
# Save the results dictionary as a NumPy binary file for future use
os.makedirs(f"./Data/{backend_name}/", exist_ok=True)
np.save(f"./Data/{backend_name}/{nq}_FC{extra}.npy", results)

In [179]:
# Get the number of nodes (qubits) in the graph problem
nq = results["G"].number_of_nodes()

# Get the number of sections (used for processing multiple groups of qubits in a single execution)
sections = results["sections"]

# Initialize dictionaries to store postprocessing results
postprocessing = {}
postprocessing_mitig = {}

# Iterate over different delta values (used for QAOA parameter tuning)
for delta in results["samples"]:
    postprocessing[delta] = {}
    postprocessing_mitig[delta] = {}

    # Iterate over different QAOA depths (p values)
    for p in results["samples"][delta]:
        print(f"----------- p = {p} -------------")
        postprocessing[delta][p] = {}
        postprocessing_mitig[delta][p] = {}

        # Iterate over different sections (to handle multiple independent executions within a job)
        for sec in range(sections):
            samples_sec = defaultdict(int)

            # Extract relevant bitstring samples for the current section
            for k, v in results["samples"][delta][p].items():
                samples_sec[k[sec*nq:(sec+1)*nq]] += v

            # Compute the MaxCut objective for the extracted samples
            postprocessing[delta][p][sec] = objective_MaxCut(samples_sec, results["G"], results["optimal"])

            # Apply error mitigation to the samples
            # new_samples = mitigate(samples_sec, results["G"], random=False)

            # Compute the MaxCut objective after error mitigation
            # postprocessing_mitig[delta][p][sec] = objective_MaxCut(new_samples, results["G"], results["optimal"])

# Store the postprocessing results
results["postprocessing"] = postprocessing
# results["postprocessing_mitig"] = postprocessing_mitig

# Generate random bitstring samples for comparison (10,000 random samples)
rand_samples = random_samples(1_000, nq)

# Compute MaxCut objective for the random samples
results["random"] = objective_MaxCut(rand_samples, results["G"], results["optimal"])

# Apply error mitigation to the random samples and compute MaxCut objective
# results["random_mitig"] = objective_MaxCut(mitigate(rand_samples, results["G"], random=False), results["G"], results["optimal"])

# Save the updated results back to a NumPy binary file
np.save(f"./Data/{backend_name}/{nq}_FC{extra}.npy", results)


----------- p = 3 -------------


In [181]:
{p:results["postprocessing"][results["Deltas"][0]][p][0]["r"] for p in results["ps"]}

{3: np.float64(0.7535529032583002)}

In [182]:
results["random"]["r"]

np.float64(0.8459046180188361)

In [99]:
# Wait for job to complete and retrieve results
qnx.jobs.wait_for(result_ref)
job_result = qnx.jobs.results(result_ref)[0].download_result()
print("Job completed successfully!")
print(f"Number of shots: {len(job_result.results)}")

results["job_id"] = str(result_ref)
# Store job ID for tracking

Job completed successfully!
Number of shots: 100


In [ ]:


# Store results in structured format (matching FC-Experiments.ipynb)
# Format: results["samples"][delta][p] = {bitstring: count}
delta_tuple = results["Deltas"][0]  # Get the delta tuple
if "samples" not in results:
    results["samples"] = {}

if delta_tuple not in results["samples"]:
    results["samples"][delta_tuple] = {}

results["samples"][delta_tuple][p] = samples_dict

# Calculate approximation ratio using the objective_MaxCut function
postprocessing = objective_MaxCut(samples_dict, graph, optimal_sol)

print(f"\nResults for p={p}, delta_beta={delta_beta}, delta_gamma={delta_gamma}:")
print(f"Approximation ratio (r): {postprocessing['r']:.4f}")
print(f"Probability of optimal: {postprocessing['probability']:.4f}")
print(f"\nTop 10 most common bitstrings:")
for bitstring, count in sorted(samples_dict.items(), key=lambda x: x[1], reverse=True)[:10]:
    energy = cost_maxcut(bitstring, {(i, j): graph[i][j]["weight"] for i, j in graph.edges})
    print(f"  {bitstring}: {count:4d} ({100*count/len(outcomes):5.1f}%) - energy: {energy:.2f}")

# Save results to file
import os
os.makedirs(f"./Data/{backend_name}", exist_ok=True)
np.save(f"./Data/{backend_name}/{nq}_FC.npy", results)
print(f"\n✓ Results saved to ./Data/{backend_name}/{nq}_FC.npy")


Approximation ratio (r): 0.9176
Probability of optimal: 0.1800


# Load and Postprocess Saved Results

Load previously saved results and perform postprocessing analysis.

In [ ]:
# Load saved results
backend_name = "helios_1e_lite"
nq = 56

results = np.load(f"./Data/{backend_name}/{nq}_FC.npy", allow_pickle=True).item()

# Get metadata
graph = results["G"]
optimal_sol = results["optimal"]
ps = results["ps"]
deltas = results["Deltas"]

print(f"Loaded results for nq={nq}, backend={backend_name}")
print(f"QAOA depths (p): {ps}")
print(f"Deltas: {deltas}")
print(f"Number of samples collected: {sum(len(results['samples'][d]) for d in results['samples'])}")

# Postprocess all results
postprocessing_all = {}
for delta in results["samples"]:
    postprocessing_all[delta] = {}
    for p in results["samples"][delta]:
        samples = results["samples"][delta][p]
        postproc = objective_MaxCut(samples, graph, optimal_sol)
        postprocessing_all[delta][p] = postproc
        
        print(f"\nDelta={delta}, p={p}:")
        print(f"  Approximation ratio (r): {postproc['r']:.4f}")
        print(f"  Probability of optimal: {postproc['probability']:.4f}")

In [ ]:
# Plot results vs p
import matplotlib.pyplot as plt

# Extract r values for each delta and p
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for delta in postprocessing_all:
    ps_sorted = sorted(postprocessing_all[delta].keys())
    r_values = [postprocessing_all[delta][p]['r'] for p in ps_sorted]
    prob_values = [postprocessing_all[delta][p]['probability'] for p in ps_sorted]
    
    delta_label = f"δ_β={delta[0]:.2f}, δ_γ={delta[1]:.2f}" if isinstance(delta, tuple) else f"δ={delta:.2f}"
    
    # Plot approximation ratio
    ax1.plot(ps_sorted, r_values, 'o-', markersize=8, linewidth=2, label=delta_label)
    
    # Plot probability of optimal
    ax2.plot(ps_sorted, prob_values, 's-', markersize=8, linewidth=2, label=delta_label)

# Format plot 1
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Random baseline')
ax1.set_xlabel('QAOA Depth (p)', fontsize=12)
ax1.set_ylabel('Approximation Ratio (r)', fontsize=12)
ax1.set_title(f'QAOA Performance vs Depth\\n{nq}-qubit FC MaxCut on {backend_name}', 
              fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)
ax1.set_ylim([0.4, 1.05])

# Format plot 2
ax2.set_xlabel('QAOA Depth (p)', fontsize=12)
ax2.set_ylabel('Probability of Optimal Solution', fontsize=12)
ax2.set_title(f'Optimal Solution Sampling vs Depth\\n{nq}-qubit FC MaxCut', 
              fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

# Print summary table
print("\n" + "="*80)
print(f"SUMMARY: {nq}-qubit FC MaxCut on {backend_name}")
print("="*80)
print(f"{'Delta':<20} {'p':<5} {'r (ratio)':<12} {'P(optimal)':<15} {'Shots':<10}")
print("-"*80)
for delta in postprocessing_all:
    delta_str = f"({delta[0]:.2f}, {delta[1]:.2f})" if isinstance(delta, tuple) else f"{delta:.2f}"
    for p in sorted(postprocessing_all[delta].keys()):
        r = postprocessing_all[delta][p]['r']
        prob = postprocessing_all[delta][p]['probability']
        n_shots = sum(postprocessing_all[delta][p]['results'][:, 2])
        print(f"{delta_str:<20} {p:<5} {r:<12.4f} {prob:<15.4f} {int(n_shots):<10}")
print("="*80)